# Pandas for spreadsheet people

This notebook is an interactive pandas tutorial written for folks who love spreadsheets.

We walk through the famous Titanic dataset to gain familiarity with data selection and inspection in `pandas`. Then, we explore  learn how to use functions, basic stats, pivot-tables, and more.

The majority of this tutorial is copied from [the pandas documentation](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/index.html).

![pandas](./static/pandas.jpeg)

## Contents

- [What is pandas?](#what-pandas)
- [Common misconceptions](#misconceptions)
- [What kind of data does pandas handle?](#what-data)
- [Reading and writing tabular data](#how-do-io)
    - [Inspecting your data](#inspect)
- [Subsetting the DataFrame](#subset)
    - [Selecting specific columns from a DataFrame](#subset-columns)
    - [Selecting specific rows from a DataFrame](#subset-rows)
    - [Selecting specific rows and columns from a DataFrame](#subset-both)
    - [Changing subsets of a DataFrame](#subset-set)
- [Creating new columns derived from existing columns](#new-columns)
    - [Applying advanced functions](#advanced-functions)
- [Summary statistics](#summary-stats)
    - [Aggregating statistics](#aggregating)
    - [Aggregating statistics grouped by category](#grouped-aggregation)
- [Next steps](#next-steps)


<a id='what-pandas'></a>
## What is `pandas`?

`pandas` is a Python package that provides fast, flexible, and expressive data structures designed to make working with "relational" or "labeled" data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, __real world__ data analysis in Python. Additionally, it has the broader goal of becoming __the most powerful and flexible open source data analysis / manipulation tool available in any language__. It is already well on its way towards this goal.

`pandas` is crowd-sourced, like the Wikipedia. It will continue to improve so long individuals continue to improve it. It is __not owned__.

In [1]:
import pandas as pd

You'll use the import statement above whenever you want to use `pandas`. __pd__ is the community-recognized alias.

<div class="alert alert-block alert-warning">

## Common misconceptions
<a id='misconceptions'></a>

Before we start, we should get some misconceptions out of the way.

- __You need to be good at numpy to use pandas__: We won't even import `numpy` in this tutorial. You can do extremely complex pandas operations knowing next-to-zero `numpy`, and that's how it should be!


- __You need to use complex loops to do complex things with DataFrame__: Not true either. If you find yourself writing complex loops to change or make calculations on data you are eventually going to make a mistake. `pandas` is designed so that you don't do this.
</div>

<a id='what-data'></a>
## What kind of data does `pandas` handle?

`pandas` works with tables, like many other tools. The primary datatype in `pandas` is the `DataFrame` which behaves just like a table.

![table_dataframe](./static/schemas/01_table_dataframe.svg)

To manually store data in a table, we can create a DataFrame with a __dictionary of lists__. When using a Python dictionary of lists, the dictionary keys will be used as column headers and the values in each list as columns of the DataFrame.



***
#### Example 1
<a id='example1'></a>

Suppose I want to store passenger data from the Titanic in a table. For a number of passengers, I know the name (characters), age (integers) and sex (male/female) data.

In [2]:
dol = {  # A dictionary of lists (DOL)
    "Name": [
        "Braund, Mr. Owen Harris",
        "Allen, Mr. William Henry",
        "Bonnell, Miss. Elizabeth",
    ],
    "Age": [22, 35, 58],
    "Sex": ["male", "male", "female"],
}

# Make our table
df = pd.DataFrame(dol) # A DataFrame (DF)
df  # You can inspect this 

,Name,Age,Sex
0,"Braund, Mr. Owen Harris",22,male
1,"Allen, Mr. William Henry",35,male
2,"Bonnell, Miss. Elizabeth",58,female


<div class="alert alert-block alert-info">

#### Exercise 1
<a id='exercise1'></a>

Try to make your own __DataFrame__ using a Python __dictionary of lists__.
</div>

In [3]:
# Try it!
# dol = ??
# df = ??
# df

`pandas` has another datatype called `Series`. A series is like a super-powered Python list. You can think of it as a single column of data.

![series](./static/schemas/01_table_series.svg)

<div class="alert alert-block alert-warning">
    
#### Warning

In a more advance sense, `Series` is an object derived from numpy's `array`. The difference is that `Series` maintains the index from its parent `DataFrame`. When you call `values` on column, you are deleting the index of the column without gaining anything.
    
If you are taking apart a DataFrame and putting it back together with numpy arrays, you'll eventually make a mistake with index alignment, which can be difficult to catch.

<a id='example2'></a>
#### Example 2

Suppose you want to work only with a single column of passenger data, like __Age__.  To select the column, use the column label in between square brackets []. The result will be a `Series`.

In [4]:
dol = {  # A dictionary of lists (DOL)
    "Name": [
        "Braund, Mr. Owen Harris",
        "Allen, Mr. William Henry",
        "Bonnell, Miss. Elizabeth",
    ],
    "Age": [22, 35, 58],
    "Sex": ["male", "male", "female"],
}

# Make our table
df = pd.DataFrame(dol) # AbDataFrame (DF)

# Select a single column (as a series)
df["Age"]

0    22
1    35
2    58
Name: Age, dtype: int64

Series are iterable like Python lists. You can build your loops with them.

In [5]:
# Iterate through column data
for age in df["Age"]:
    print(age)

22
35
58


Series support a lot of functions you would find in your spreadsheet program.

In [6]:
# Get the maximum age in our data
df["Age"].max()

58

<div class="alert alert-block alert-info">

#### Exercise 2
<a id='exercise2'></a>
Using your DataFrame from [Exercise 1](#exercise1), try to select a __Column__.
</div>

In [7]:
## Try it!
##
##

<a id='how-do-io'></a>
## How do I read and write tabular data?

`pandas` provides the `read_csv()` function to read data stored as a csv file into a pandas DataFrame. pandas supports many different file formats or data sources out of the box (csv, excel, sql, json, parquet, …), each of them with the prefix __read_*__.

![io](./static/schemas/02_io_readwrite.svg)

We'll read in a dataset from the Titanic using `read_csv()`. It can work on files from your computer and files on the web. 

- __Survived__: 1 for yes, 0 for no
- __Pclass__: The passenger's class, (1st, 2nd, or 3rd)
- __Name__: The passenger's recorded name
- __Sex__: Sex of passenger
- __Age__: Age of passenger (float because there were some young ones)
- __SibSp__: Number of the passenger's siblings aboard
- __Parch__: Number of the passenger's parents/children aboard
- __Ticket__: Ticket associated with the passenger
- __Fare__: Ticket fare (in pounds)
- __Cabin__: Passenger's assigned cabin
- __Embarked__: Port where the passenger got on the boat. C = Cherbourg, Q = Queenstown, 
S = Southampton


In [8]:
# Import titanic dataset
# You can use read_csv with any file on the computer as well as files from the web
# We specify that one of the columns should be used for the DataFrame's index
titanic_datapath = "https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv"
titanic = pd.read_csv(titanic_datapath, index_col="PassengerId")
titanic

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


<a id='example3'></a>
#### Example 3

My boss requested the Titanic data as a spreadsheet. You can send the DataFrame to a CSV file (which can be opened in a spreadsheet program) using the `to_csv()` method. 

In [9]:
## Save to file. The directory must exist for this to work. Have a file name handy!
titanic.to_csv('./titanic.csv')  # Will save it in this notebook's directory

<div class="alert alert-block alert-info">

#### Exercise 3
<a id='exercise3'></a>

Try to save the titanic DataFrame as a CSV file somewhere on your computer. Then, open it with Excel.
Here's a hint, you can find your computer's home directory (Has Desktop, Documents, etc.) with the following Python code:
</div> 
   

```python
from pathlib import Path
print(Path.home())
```

In [10]:
## Try it!
##
##

<a id='inspect'></a>
### How do I inspect data?

Whenever you have a lot of information in front of you, it's a good idea to inspect it. We'll look at a few `pandas` functions for inspecting data we load in.
***
<a id='example4'></a>
#### Example 4

I want to see the first 8 rows of the titanic DataFrame. You can use the `head()` method to view the first few rows of a dataset. You can use the `tail()` method to view the last few rows of a DataFrame.

In [11]:
# Using head to view the top rows of a dataframe
titanic.head(8)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S


In [12]:
# Using tail to view the bottom rows of a dataframe
titanic.tail(8)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.500,NaN,S
885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.050,NaN,S
886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.125,NaN,Q
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.450,NaN,S
890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.000,C148,C
891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.750,NaN,Q


You may want to check out the datatypes in the DataFrame. You can do that with `dtypes`.

In [13]:
# Look at the dtypes attribute to check on the DataFrame's data types.
titanic.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

For a technical summary of the dataframe, use `info()`.

In [14]:
# info() tells dtypes, how much data is missing, and size in memory
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


To see the unique values in a column, use `unique()`.

In [15]:
titanic["Pclass"].unique() # Good, it's like it was in the movie

array([3, 1, 2], dtype=int64)

<a id='subset'></a>
## How do I select and change a subset of a `DataFrame`?

Datasets can be big with lots of features. Ideally, you can target portions of them when you run
manipulations and conversions. This portion of the tutorial shows you how to select subsets of your DataFrames.

In a spreadsheet program, it's straightforward to select rows, columns, and individual cells. But
what if you need to make selections and/or changes based on the __content__ of the cells? In `pandas` we can do this with __Boolean Indexing__.

***
<a id='subset-columns'></a>
### How do I select specific columns from a `DataFrame`?

We've seen how to select a single column as a series, but you can also select multiple columns, returning a smaller DataFrame.

![subset_columns](./static/schemas/03_subset_columns.svg)

<a id='example5'></a>
#### Example 5

We would like to select the __Age__ and __Sex__ of all passengers on the titanic. We can do that by selecting a list of column names from the DataFrame. This will return a new DataFrame containing only the columns we've selected.

In [16]:
# Selecting multiple columns, using a list of column names
age_sex = titanic[["Age", "Sex"]]
age_sex.head()

,Age,Sex
PassengerId,,
1,22.0,male
2,38.0,female
3,26.0,female
4,35.0,female
5,35.0,male


<div class="alert alert-block alert-info">

#### Exercise 4
<a id='exercise4'></a>

Try to select the __Parch__, __SibSp__, and __Fare__ columns from `titanic_dataset` DataFrame.
</div>

In [17]:
## Try it!
##
##

<a id='subset-rows'></a>
### How do I select specific rows from a `DataFrame`?

You can use __Boolean Indexing__ to select and filter specific rows from the DataFrame. In __Boolean Indexing__, you use boolean operators to tell pandas to subset based on a defined criteria.

![subset_rows](./static/schemas/03_subset_rows.svg)

#### Operators
- == : Is equal to
- \> : Greater than
- < : Less than
- \>= : Greater than or equal
- <= : Less than or equal
- != : not equal

***
<a id='example6'></a>
#### Example 6

I want a new DataFrame that is a subset of the titanic dataset that has only passengers over the age of 35.

In [18]:
# Selecting passengers over 35 with boolean indexing
over_35 = titanic[titanic["Age"] > 35]
over_35

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S
16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
866,1,2,"Bystrom, Mrs. (Karolina)",female,42.0,0,0,236852,13.0000,NaN,S
872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
874,0,3,"Vander Cruyssen, Mr. Victor",male,47.0,0,0,345765,9.0000,NaN,S


If you need a more specific selection, we can use binary operators to come up with a __Selection Criteria__ to use in __Boolean Indexing__.

#### Binary operators:
- & : and, (criteria1 & criteria2)
- | : or, (criteria1 | criteria2)
- ~ : complement (~criteria)
- ^ : XOR (criteria1 ^ criteria1)

***
<a id='example7'></a>
#### Example 7

I'm interested in male passengers from the 1st class.

In [19]:
# Building a boolean criteria for men in the 1st class and using it in Boolean Indexing
criteria = (titanic["Sex"] == "male") & (titanic["Pclass"] == 1)
rich_guys = titanic[criteria]
rich_guys

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5000,A6,S
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,C23 C25 C27,S
31,0,1,"Uruchurtu, Don. Manuel E",male,40.0,0,0,PC 17601,27.7208,NaN,C
35,0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...
840,1,1,"Marechal, Mr. Pierre",male,NaN,0,0,11774,29.7000,C47,C
858,1,1,"Daly, Mr. Peter Denis",male,51.0,0,0,113055,26.5500,E17,S
868,0,1,"Roebling, Mr. Washington Augustus II",male,31.0,0,0,PC 17590,50.4958,A24,S


<div class="alert alert-block alert-info">

#### Exercise 5
<a id='exercise5'></a>

Use __Boolean Indexing__ to make a new `DataFrame` that is a subset of `titanic_dataset`. Choose something interesting, like dead/alive, passenger class, deck, etc.
</div>

In [20]:
## Try it!
#
#

<a id='subset-both'></a>
### How do I select specific rows and columns from a `DataFrame`?

If you want to select specific rows and columns in one shot (for example, to change a columns for rows only matching a particular criteria) you must use `loc` or `iloc`. These operators use the same kinds of selectors we've already discussed, but they also allow you to use both a selector for rows and a selector for columns.

![subset_rows](./static/schemas/03_subset_columns_rows.svg)

<a id='example8'></a>
#### Example 8

I'm interested in the names of the passengers older than 35 years.

In [21]:
# Using loc to select the names of passengers older than 35
older_35_names = titanic.loc[titanic["Age"] > 35, "Name"]
older_35_names

PassengerId
2      Cumings, Mrs. John Bradley (Florence Briggs Th...
7                                McCarthy, Mr. Timothy J
12                              Bonnell, Miss. Elizabeth
14                           Andersson, Mr. Anders Johan
16                      Hewlett, Mrs. (Mary D Kingcome) 
                             ...                        
866                             Bystrom, Mrs. (Karolina)
872     Beckwith, Mrs. Richard Leonard (Sallie Monypeny)
874                          Vander Cruyssen, Mr. Victor
880        Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)
886                 Rice, Mrs. William (Margaret Norton)
Name: Name, Length: 217, dtype: object

<a id='example9'></a>
#### Example 9

I'm interested in rows 10 till 25 and columns 3 to 5. You can use `iloc` to select specific rows and columns manually. It works like a click+drag selection in your spreadsheet program.

In [22]:
# Using iloc: dataframe.iloc[(row_start:row_end, column_start:column_end)]
selection = titanic.iloc[9:25, 2:5]
selection

,Name,Sex,Age
PassengerId,,,
10,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0
11,"Sandstrom, Miss. Marguerite Rut",female,4.0
12,"Bonnell, Miss. Elizabeth",female,58.0
13,"Saundercock, Mr. William Henry",male,20.0
14,"Andersson, Mr. Anders Johan",male,39.0
15,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0
16,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0
17,"Rice, Master. Eugene",male,2.0
18,"Williams, Mr. Charles Eugene",male,NaN


<div class="alert alert-block alert-info">
    

#### Exercise 6
<a id='exercise6'></a>

Use `loc` to get the fares of all passengers in the 1st class. Or, try to use a __selection criteria__ to find out something more interesting about the titanic dataset.
    
</div>

In [23]:
## Try it!
#
#

<a id='subset-set'></a>
### How do I change values on a subset of a `DataFrame`?

When selecting specific rows and/or columns with `loc` or `iloc`, new values can be assigned to the selected data. 

I strongly recommend you make a copy of your dataframe before doing this, in case you make a mistake. That way you don't have to start over from your first cell in the notebook! (not a big deal if you have to)

The ability to copy a DataFrame in `pandas` is a huge advantage over your spreadsheet program.

***
<a id='example10'></a>
#### Example 10

I'd like to change the names of all male passengers in the first class to "rich dude". First, we make a copy of the dataset so we can always refer back to the original data. Second, we use __Boolean Indexing__ to select male passengers from the first class. Finally, we set their names.

In [24]:
# Changing data with loc and Boolean Indexing
copied_dataset = titanic.copy()  # Make copy
criteria = (copied_dataset["Sex"] == "male") & (copied_dataset["Pclass"] == 1)  # Build criteria
copied_dataset.loc[criteria, "Name"] = "rich_dude"  # Setting the names
copied_dataset.loc[copied_dataset["Sex"] == "male"]  # Inspecting our changes

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,rich_dude,male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S


<div class="alert alert-block alert-info">

#### Exercise 7
<a id='exercise7'></a>

Use `loc` to make a selection from titanic dataset and change some values. Be sure to copy `titanic` before making changes. 

</div>

In [25]:
## Try it!
#
#

<a id="new-columns"></a>
## How do I create new columns derived from existing columns?

In a spreadsheet program, you can set column data to be the output of a function of other column data. You can do this in `pandas` with even more control.

![newcolumn](./static/schemas/05_newcolumn_2.svg)

<a id='example11'></a>
#### Example 11

I'd like to know the family size of all passengers on the titanic. We can get this by adding up the values in the `Parch` and `SibSp` columns.

In [26]:
family_data = titanic.copy()
family_data["family_size"] = family_data["SibSp"] + family_data["Parch"]
family_data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,family_size
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,3


<div class="alert alert-block alert-info">

#### Exercise 8
<a id='exercise8'></a>

Assume that only a single family member pays the ticket fare to board the titanic. This fare is the same in our dataset for each member of the family. It may be more useful to know the __per-person-fare__.

$$ppf = \frac{Fare}{(Parch + SibSp + 1)}$$

Copy the DataFrame into a new DataFrame, the use column functions to create a `ppf` column that implements the equation above.
</div>

In [27]:
## Try it!
#
#

<a id="advanced-functions"></a>

### Applying advanced functions
For more advanced functions using data from multiple columns, we need to use the `apply` method of the DataFrame. For example, we could calculate the per-person-fare using the following syntax. Although trickier to implement, this will scale better and give us the best performance.

```python
def ppf(fare, parch, sibsp):
    return fare / (parch + sibsp + 1)
    
titanic['ppf'] = titanic[['Fare', 'Parch', 'SibSp']].apply(lambda x: ppf(*x), axis=1)
```

This block of code will require more time than we have in-workshop to explain, so if you are curious about how this works, please reach out!

<a id="summary-stats"></a>
## Summary statistics

Different statistics are available and can be applied to columns with numerical data. By default, operations exclude missing data and operate across rows.

![aggregation](./static/schemas/06_reduction.svg)

<a id='aggregating'></a>
### Aggregating statistics

<a id='example12'></a>
#### Example 12

What is the mean age and ticket fare price of the Titanic passengers? This is straightforward to do, we just select the columns we want the mean for and call mean on them. 

Try selecting a single column only for the mean and see what happens.

In [28]:
# Calling a summary function on multiple columns
titanic[["Age", "Fare"]].mean()

Age     29.699118
Fare    32.204208
dtype: float64

`pandas` supplies another incredible method `describe()`, which will give us more stats on our column data.

In [29]:
titanic[["Age", "Fare"]].describe()

,Age,Fare
count,714.000000,891.000000
mean,29.699118,32.204208
std,14.526497,49.693429
min,0.420000,0.000000
25%,20.125000,7.910400
50%,28.000000,14.454200
75%,38.000000,31.000000
max,80.000000,512.329200


<div class="alert alert-block alert-info">

#### Exercise 9
<a id='exercise9'></a>

Using your answer from [Exercise 8](#exercise8) and aggregating stats, obtain the maximum family size on the Titanic (or anything else you want to see).
    
</div>

In [30]:
## Try it!
#
#

<a id='grouped-aggregation'></a>
### Aggregating statistics grouped by category

Statistics on columns is great, but we can get more precise with our aggregations. Calculating a given statistic (e.g. mean age) for each category in a column (e.g. male/female in the Sex column) is a common pattern. The `groupby` method is used to support this type of operations.

You might recognize this as _pivot table_ from your favorite spreadsheet program.

![groupby](./static/schemas/06_groupby.svg)

<a id='example13'></a>
#### Example 13

From what we saw in the movie, we may assume that a higher proportion of females than males survived the disaster. If we use `groupby`, we can group the passengers by sex, then assess the mean survival of each group.

In [31]:
titanic.groupby("Sex")["Survived"].mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

<a id='example14'></a>
#### Example 14

You can group by multiple categories. Does cabin class correlate with survival?

In [32]:
titanic.groupby(["Pclass", "Sex"])["Survived"].mean()  # yeah, it's correlated

Pclass  Sex   
1       female    0.968085
        male      0.368852
2       female    0.921053
        male      0.157407
3       female    0.500000
        male      0.135447
Name: Survived, dtype: float64

<div class="alert alert-block alert-info">

#### Exercise 10
<a id='exercise10'></a>

Figure out the mean ticket fare for each cabin class using `groupby`.
    
</div>

In [33]:
## Try it!
#
#

<a id="next-steps"></a>
## Next Steps

This tutorial only scratches the surface of `pandas`, covering the operations and functions that most users will find useful. As you start using `pandas` for your data analysis, you will learn more about the advanced features.

Here are a few places you may want to go next.

- [Make interactive graphs with pandas and plotly](https://plotly.com/python/line-and-scatter/)
- [Reshape data](https://pandas.pydata.org/docs/getting_started/intro_tutorials/07_reshape_table_layout.html)
- [Aggregate data from multiple sources](https://pandas.pydata.org/docs/getting_started/intro_tutorials/08_combine_dataframes.html)